In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

# Orgaize data

In [2]:
stocks = pd.read_csv('starter_files/sp_500_stocks.csv')


Acquiring an API Token
IEX cloud 

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data['symbol'])

AAPL


# Parsing Our Api Call
API call of IEx

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

# Adding Stocks Data to a Pandas DataFrame

In [6]:
my_cols = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Share to Buy']
final_dataframe = pd.DataFrame(columns = my_cols)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Share to Buy


In [7]:
final_dataframe.append(

    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
      index = my_cols
    ),
    ignore_index=True

)

,Ticker,Stock Price,Market Capitalization,Number of Share to Buy
0,AAPL,129.07,2129329462505,N/A


# Looping Through The Tickers in our List of Stocks

In [8]:
final_dataframe = pd.DataFrame(columns =  my_cols)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            
            [
                
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
                
            ],
            index = my_cols
        ),
        ignore_index = True
    
    )
    

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Share to Buy
0,A,126.48,2181027487557,N/A
1,AAL,126.38,2129004797224,N/A
2,AAP,129.21,2168032118647,N/A
3,AAPL,130.10,2201466206356,N/A
4,ABBV,127.60,2125233765559,N/A


# Optimize the API call

In [10]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_cols)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                    
                ],
            
            index = my_cols
            ),
            ignore_index=True
        
        )
        
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Share to Buy
0,A,132.64,41862037286,N/A
1,AAL,22.58,14317776373,N/A
2,AAP,204.32,13871977874,N/A
3,AAPL,126.42,2188331106203,N/A
4,ABBV,119.71,206300144785,N/A
...,...,...,...,...
500,YUM,123.55,36181008143,N/A
501,ZBH,171.58,35184588513,N/A
502,ZBRA,486.46,26105605582,N/A
503,ZION,60.10,9825569975,N/A


# Calculating Number of Shares to Buy

In [12]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \nPlease Try again")
    portfolio_size = input('Enter the value of your portfolio:')



Enter the value of your portfolio:1000000
1000000.0


In [18]:
position_size  = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])

    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Share to Buy,Number of Shares to Buy
0,A,132.64,41862037286,N/A,14.0
1,AAL,22.58,14317776373,N/A,87.0
2,AAP,204.32,13871977874,N/A,9.0
3,AAPL,126.42,2188331106203,N/A,15.0
4,ABBV,119.71,206300144785,N/A,16.0
...,...,...,...,...,...
500,YUM,123.55,36181008143,N/A,16.0
501,ZBH,171.58,35184588513,N/A,11.0
502,ZBRA,486.46,26105605582,N/A,4.0
503,ZION,60.10,9825569975,N/A,32.0


# XLS writer now

In [20]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)